## Use NLP to discover the current product trends

### Topic modeling visualization

In [1]:
%pylab inline

import pandas as pd
import numpy as np
import pickle as pk
from scipy import sparse as sp
import re

Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('all.csv')
df1 = df[df.website =='ASOS']
docs = df1['alltext']
df.sample(5)

,Unnamed: 0,alltext,brand,category,description,img_2,img_id,img_url,material,price,product_name,website,id
234,234,T-shirt with Printed Design T-shirt in soft co...,HM,Top,T-shirt in soft cotton jersey with a printed m...,NaN,234 T-shirt with Printed Design,https://lp2.hm.com/hmgoepprod?set=source[/aa/b...,Cotton 100%,$12.99,T-shirt with Printed Design,HM,234
621,621,"A-line Jersey Dress Short, sleeveless A-line d...",HM,Dress,"Short, sleeveless A-line dress in jersey.",NaN,132 A-line Jersey Dress,https://lp2.hm.com/hmgoepprod?set=source[/a1/1...,Viscose 100%,$12.99,A-line Jersey Dress,HM,621
950,950,Jersey Dress Straight-cut dress in thick jerse...,HM,Dress,Straight-cut dress in thick jersey with 3/4-le...,NaN,461 Jersey Dress,https://lp2.hm.com/hmgoepprod?set=source[/07/8...,Viscose 65%; Polyester 30%; Elastane 5%,$29.99,Jersey Dress,HM,950
9264,9264,Warehouse textured button through dress in nav...,NaN,Dress,"Do you believe in love at first sight?,High ne...",NaN,NaN,https://images.asos-media.com/products/warehou...,"Breathable cotton,What's cooler than being coo...",$67.00,Warehouse textured button through dress in navy,ASOS,9264
5899,5899,ASOS 4505 scoop neck racer back bra Witness th...,NaN,Top,"Witness the fitness,Scoop neck,Non-padded cups...",NaN,NaN,https://images.asos-media.com/products/asos-45...,"Smooth stretch fabric,Ticks all your comfort b...",$26.00,ASOS 4505 scoop neck racer back bra,ASOS,5899


In [3]:
print(docs.index)
docs.head()

Int64Index([ 3136,  3137,  3138,  3139,  3140,  3141,  3142,  3143,  3144,
             3145,
            ...
            10205, 10206, 10207, 10208, 10209, 10210, 10211, 10212, 10213,
            10214],
           dtype='int64', length=7079)


3136    ASOS DESIGN blouse with frill shoulder Lightwe...
3137    A Star Is Born Going Out Festival High Neck Bo...
3138    Converse Cons Skate Boarding Long Sleeve T-Shi...
3139    Vans White Finish Line Heritage Long Sleeve T-...
3140    ASOS DESIGN x glaad& Curve t-shirt with high n...
Name: alltext, dtype: object

### Pre-process and vectorize the documents

In [4]:
from nltk.tokenize import RegexpTokenizer

def preprocessDoc(docs):
    docs = np.array(docs)  # no longer have index
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        
        stop = ['ASOS', 'macy', 'bloomingdale', "fashion nova", "YAS", "Ditsy", "Noisy", "May", "Ted","Baker", "River","Island", "Karen","Scott","PrettyLittleThing","Roxy","DESIGN","Chi", \
               "Alfani","Boohoo","Sofie","Schnoor","Ellesse", "Jeannie","TFNC","Sacred", "Hawk","Urban","Bliss","Puma","adidas", "Stella", \
               'cm', 'size', 'web id', 'approx', 'model', 'height', 'is', 'and', 'she', 'wearing', 'small', 'approximate', 'measurements', 'height', 
               'bust', 'waist', 'hips', 'made', 'usa', 'things', 'regular', 'right', 'kind', 'that', 'things']

        lst = r'|'.join([x.lower() for x in stop])

        docs[idx] = re.sub(lst,'',docs[idx])
        
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    docs = [[token for token in doc if not token.isdigit()] for doc in docs]  #remove numbers
    docs = [[token for token in doc if len(token) > 3] for doc in docs] #remove words that are short 
  
    return docs

docs = preprocessDoc(docs)

#### Compute bigrams/trigrams and remove less words and very common words:

In [5]:
from gensim.corpora import Dictionary
from gensim.models import Phrases
bigram = Phrases(docs, min_count=10, threshold=10)  # only ones that appear 10 times or more.
trigram = Phrases(bigram[docs])

for i in range(len(docs)):
    for token in bigram[docs[i]]:
        if '_' in token:
            docs[i].append(token)
            print ("bi:", token)
    for token in trigram[docs[i]]:
        if token.count('_') > 2 :
            docs[i].append(token)
            print ("tri:", token)

dictionary_ = Dictionary(docs)
print (len(dictionary_))
dictionary_.filter_extremes(no_below=10, no_above=0.2)
print (len(dictionary_))

/Users/kellyho/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


bi: button_placket
bi: fancy_pants
bi: press_stud
bi: fastening_underneath
bi: close_bodycon
bi: exclusive_heard
bi: here_first
bi: soft_touch
tri: press_stud_fastening_underneath
tri: exclusive_heard_here_first
bi: long_sleeve
bi: bigger_bring
bi: soft_touch
bi: long_sleeve
bi: nothing_beats
bi: fresh_crew
bi: logo_lifestyle
bi: loose_tight
bi: soft_touch
bi: other_words
tri: nothing_beats_fresh_crew
bi: glaad_support
bi: movement_glaad
bi: works_accelerate
bi: lgbtq_acceptance
bi: create_meaningful
bi: change_every
bi: sold_proceeds
bi: will_dona
bi: soft_touch
bi: jersey_complicate
tri: glaad_support_movement_glaad
tri: works_accelerate_lgbtq_acceptance
tri: create_meaningful_change_every
tri: sold_proceeds_will_dona
bi: never_have
bi: many_crew
bi: loose_tight
bi: soft_rounder
tri: never_have_many_crew
bi: motivation_starts
bi: quick_drying
bi: sweat_wicking
bi: holds_close
bi: uses_climalite
bi: sweat_away
bi: from_skin
bi: helps_keep
tri: sweat_away_from_skin
bi: long_sleeve
bi: 

bi: bump_baby
bi: part_edit
bi: caring_planet
bi: know_like
bi: good_stuff
bi: supermarket_uses
bi: less_pesticides
bi: grow_which
bi: means_happier
bi: healthier_planet
tri: double_layer_ideal_nursing
tri: covered_from_bump_baby
tri: part_edit_caring_planet
tri: know_like_good_stuff
tri: supermarket_uses_less_pesticides
tri: grow_which_means_happier
bi: button_placket
bi: three_quarter
bi: from_bump
bi: drapes_softly
bi: south_beach
bi: never_have
bi: many_crew
bi: dropped_shoulders
bi: relaxed_loose
bi: soft_touch
bi: comfort_aren
bi: mutually_exclusive
tri: never_have_many_crew
tri: comfort_aren_mutually_exclusive
bi: clashing_prints
bi: frill_seekers
bi: relaxed_loose
bi: slightly_want
bi: reveal_much
tri: slightly_want_reveal_much
bi: wardrobe_staple
bi: point_collar
bi: press_stud
bi: slim_narrow
bi: sits_close
bi: denim_actually
tri: slim_narrow_sits_close
bi: wardrobe_staple
bi: dropped_shoulders
bi: plain_goes
bi: stard_classic
bi: ticks_your
bi: comfort_boxes
tri: ticks_your_

bi: some_serious
bi: daytime_inspiration
bi: cami_straps
bi: dropped_armholes
bi: plain_goes
bi: relaxed_loose
bi: soft_touch
bi: ticks_your
bi: comfort_boxes
tri: some_serious_daytime_inspiration
tri: ticks_your_comfort_boxes
bi: slacker_staple
bi: drawstring_hood
bi: dropped_shoulders
bi: more_room
bi: soft_touch
bi: great_buddy
tri: slacker_staple_drawstring_hood
bi: cropped_length
bi: slim_narrow
bi: sits_close
bi: jersey_complicate
tri: slim_narrow_sits_close
bi: striped_another
bi: staple_line
bi: like_dream
tri: striped_another_staple_line
bi: text_appeal
bi: cropped_length
bi: soft_rounder
bi: some_serious
bi: daytime_inspiration
bi: ticks_your
bi: comfort_boxes
bi: metalled_fibres
tri: some_serious_daytime_inspiration
tri: ticks_your_comfort_boxes
bi: reclaimed_vintage
bi: jersey_complicate
bi: long_sleeve
bi: your_sartorial
bi: rainbow_stripe
bi: easy_wear
bi: does_what
bi: wardrobe_addition
bi: plain_understa
bi: jersey_complicate
bi: crooked_tongues
bi: soft_touch
bi: comfo

bi: cropped_length
bi: doesn_stretch
bi: nothing_beats
bi: fresh_crew
bi: short_sleeves
bi: relaxed_loose
bi: soft_touch
tri: nothing_beats_fresh_crew
bi: polka_hits
bi: cropped_length
bi: conscious_holds
bi: soft_touch
bi: keep_cool
bi: greater_range
bi: loose_falls
bi: loosely_over
tri: loose_falls_loosely_over
bi: dropped_shoulders
bi: relaxed_loose
bi: doesn_stretch
bi: fresh_threads
bi: soft_touch
bi: jersey_complicate
bi: dropped_armholes
bi: relaxed_sloung
bi: everyday_shirt
bi: basic_best
bi: never_have
bi: many_crew
bi: slogan_make
bi: soft_touch
bi: jersey_complicate
tri: never_have_many_crew
bi: clumsy_ones
bi: short_sleeves
bi: jersey_complicate
bi: cami_straps
bi: plain_goes
bi: button_keyhole
bi: relaxed_loose
bi: soft_touch
bi: doesn_stretch
bi: lace_beads
bi: beaded_embellhment
bi: cropped_length
bi: more_definitely
bi: sometimes_less
bi: press_stud
bi: fastening_underneath
bi: conscious_holds
tri: press_stud_fastening_underneath
bi: dropped_shoulders
bi: cropped_length

...

bi: doesn_stretch
tri: slim_narrow_sits_close
bi: case_needed
bi: length_between
bi: soft_touch
bi: slim_narrow
bi: sits_close
bi: more_definitely
tri: slim_narrow_sits_close
bi: short_sleeves
bi: relaxed_think
bi: baggy_casual
tri: relaxed_think_baggy_casual
bi: clumsy_ones
bi: short_sleeves
bi: length_between
bi: soft_touch
bi: jersey_complicate
bi: patterns_bold
bi: stard_classic
bi: drapes_softly
bi: adjustable_buckle
bi: functional_pockets
bi: denim_actually
bi: clumsy_ones
bi: button_placket
bi: side_pockets
bi: soft_touch
bi: jersey_complicate
bi: short_sleeves
bi: button_placket
bi: loose_tight
bi: drapes_softly
bi: wardrobe_staple
bi: needs_introduction
bi: didn_write
bi: relaxed_loose
bi: soft_touch
bi: other_words
tri: wardrobe_staple_needs_introduction
bi: welcome_casual
bi: time_shine
bi: stard_classic
bi: jersey_nail
bi: animal_yeah
bi: wild_thing
bi: short_sleeves
bi: stard_classic
bi: what_cooler
bi: than_being
tri: animal_yeah_wild_thing
tri: what_cooler_than_being
bi:

bi: drapes_softly
bi: bolder_better
bi: relaxed_loose
bi: crepe_lightly
bi: crimped_like
tri: crepe_lightly_crimped_like
bi: break_blooms
bi: adjustable_straps
bi: from_bump
bi: soft_touch
bi: drapes_softly
bi: your_plans
bi: fully_lined
bi: crossover_front
bi: stard_classic
bi: 18th_century
tri: your_plans_fully_lined
bi: best_dressed
bi: more_definitely
bi: tulle_very
bi: fine_netting
tri: tulle_very_fine_netting
bi: surpre_your
bi: monochrome_palette
bi: fully_lined
bi: tape_secure
bi: stard_classic
bi: doesn_stretch
tri: surpre_your_monochrome_palette
bi: fully_lined
bi: stard_classic
bi: drapes_softly
bi: conscious_holds
bi: thick_scuba
bi: super_smooth
bi: feel_snorkel
tri: super_smooth_feel_snorkel
bi: maternity_nursing
bi: ideal_nursing
bi: life_easier
bi: loose_tight
bi: from_bump
bi: baby_beyond
bi: soft_touch
tri: from_bump_baby_beyond
bi: vero_moda
bi: some_serious
bi: daytime_inspiration
bi: cami_straps
bi: button_placket
bi: blooming_lovely
bi: drapes_softly
tri: some_ser

bi: stard_classic
bi: soft_touch
bi: some_really
bi: cami_straps
bi: relaxed_loose
bi: doesn_stretch
bi: covered_from
bi: bump_baby
bi: soft_touch
bi: jersey_complicate
tri: covered_from_bump_baby
bi: bolder_better
bi: button_placket
bi: floaty_floaty
bi: believe_love
bi: first_sight
bi: pretty_none
bi: drapes_softly
tri: believe_love_first_sight
bi: ultimate_squad
bi: short_sleeves
bi: sequin_embellhment
bi: more_definitely
bi: tulle_very
bi: fine_netting
tri: sequin_embellhment_more_definitely
tri: tulle_very_fine_netting
bi: little_mtress
bi: some_days
bi: call_little
bi: loose_tight
bi: drapes_softly
tri: some_days_call_little
bi: little_mtress
bi: best_dressed
bi: adjustable_straps
bi: lace_inserts
bi: beaded_embellhment
bi: tulle_very
bi: fine_netting
tri: tulle_very_fine_netting
bi: little_mtress
bi: fancier_plans
bi: sequin_embellhment
bi: doesn_stretch
bi: little_mtress
bi: thing_have
bi: padded_cups
bi: from_bump
bi: fine_mesh
bi: little_mtress
bi: thing_have
bi: stard_classi

bi: crochet_lace
bi: look_without
tri: crochet_lace_look_without
bi: fully_lined
bi: what_inside
bi: length_between
bi: crepe_lightly
bi: crimped_like
tri: fully_lined_what_inside
tri: crepe_lightly_crimped_like
bi: about_beach
bi: take_dive
bi: slim_narrow
bi: sits_close
bi: soft_touch
tri: slim_narrow_sits_close
bi: long_sleeve
bi: maternity_nursing
bi: pretty_none
bi: ideal_nursing
bi: your_multitasking
bi: from_bump
bi: soft_touch
bi: ticks_your
bi: comfort_boxes
tri: ideal_nursing_your_multitasking
tri: ticks_your_comfort_boxes
bi: thing_have
bi: fully_lined
bi: short_sleeves
bi: stard_classic
bi: from_bump
bi: floaty_floaty
tri: thing_have_fully_lined
bi: animal_lead
bi: drapes_softly
bi: some_serious
bi: daytime_inspiration
bi: animal_lead
bi: drapes_softly
tri: some_serious_daytime_inspiration
bi: believe_love
bi: first_sight
bi: crossover_front
bi: drapes_softly
tri: believe_love_first_sight
bi: vero_moda
bi: adjustable_buckle
bi: functional_pockets
bi: soft_touch
bi: goes_lon

3133
1518


### Vectorize data

- bag-of-words : frequency of words

In [6]:
corpus = [dictionary_.doc2bow(doc) for doc in docs]
print('unique tokens: %d' % len(dictionary_)) #gensim.corpora.dictionary.Dictionary
print('Number of records: %d' % len(corpus)) #List

unique tokens: 1518
Number of records: 7079


## Train LDA model

In [7]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 6
chunksize = 500 #important #number of documents to be used in each training chunk. (how many documents to load into memory)
passes = 20 #important #number of passes through the corpus during training
iterations = 400 #maximum number of iterations through the corpus when inferring the topic distribution of a corpus
eval_every = 1  #log perplexity is estimated every that many updates. Setting this to one slows down training by ~2x.

# Make a index to word dictionary.
dictionary_[0]  #this is only to "load" the dictionary. Otherwise, you will get the value error. 
id2word = dictionary_.id2token

model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, alpha='auto', eta='auto', iterations=iterations, num_topics=num_topics, passes=passes, eval_every=eval_every)


In [8]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [9]:
pyLDAvis.gensim.prepare(model, corpus, dictionary_)

/Users/kellyho/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.169107 -0.189469       1        1  29.977217
3      0.211476  0.085417       2        1  17.513855
4     -0.036916  0.197289       3        1  14.855464
5      0.092805 -0.258279       4        1  13.661860
1     -0.299201  0.104001       5        1  13.114985
2      0.200943  0.061041       6        1  10.876613, topic_info=     Category         Freq               Term        Total  loglift  logprob
term                                                                        
1     Default  1718.000000             button  1718.000000  30.0000  30.0000
339   Default   773.000000              short   773.000000  29.0000  29.0000
5     Default   825.000000            placket   825.000000  28.0000  28.0000
140   Default  1540.000000             lining  1540.000000  27.0000  27.0000
320   Default  1354.000000             floral  1354.000000  26.0000  26.0000
12    Default  1218.000000              close  1218.000000  25.0000  25.0000
278   Default   755.000000              lined   755.000000  24.0000  24.0000
2     Default   754.000000     button_placket   754.000000  23.0000  23.0000
48    Default  1249.000000               crew  1249.000000  22.0000  22.0000
548   Default   624.000000      short_sleeves   624.000000  21.0000  21.0000
257   Default   638.000000          conscious   638.000000  20.0000  20.0000
228   Default  1133.000000               lace  1133.000000  19.0000  19.0000
136   Default   621.000000              holds   621.000000  18.0000  18.0000
10    Default   595.000000            bodycon   595.000000  17.0000  17.0000
212   Default   832.000000              overd   832.000000  16.0000  16.0000
19    Default  1415.000000          fastening  1415.000000  15.0000  15.0000
579   Default   899.000000               mini   899.000000  14.0000  14.0000
188   Default   866.000000               wrap   866.000000  13.0000  13.0000
82    Default   589.000000         complicate   589.000000  12.0000  12.0000
258   Default   536.000000    conscious_holds   536.000000  11.0000  11.0000
93    Default   500.000000  jersey_complicate   500.000000  10.0000  10.0000
333   Default   881.000000            relaxed   881.000000   9.0000   9.0000
248   Default   794.000000             ruffle   794.000000   8.0000   8.0000
1135  Default   741.000000               maxi   741.000000   7.0000   7.0000
407   Default   508.000000            dropped   508.000000   6.0000   6.0000
3     Default   438.000000             collar   438.000000   5.0000   5.0000
105   Default   673.000000            striped   673.000000   4.0000   4.0000
7     Default   488.000000          shoulders   488.000000   3.0000   3.0000
336   Default   744.000000             softly   744.000000   2.0000   2.0000
328   Default   739.000000      drapes_softly   739.000000   1.0000   1.0000
...       ...          ...                ...          ...      ...      ...
469    Topic6   120.489975         shot_print   120.675064   2.2170  -4.9584
468    Topic6   120.489975               shot   120.675064   2.2170  -4.9584
1353   Topic6   115.984932         bridesmaid   116.168098   2.2170  -4.9965
1412   Topic6   109.509903               moda   109.693748   2.2169  -5.0540
1414   Topic6   109.509903          vero_moda   109.693748   2.2169  -5.0540
1413   Topic6   109.509903               vero   109.693748   2.2169  -5.0540
1035   Topic6   108.645950              cutie   108.829376   2.2169  -5.0619
1041   Topic6   108.645950         take_cutie   108.829376   2.2169  -5.0619
122    Topic6    99.368248            rounder    99.558235   2.2166  -5.1511
123    Topic6    94.887299       soft_rounder    95.077019   2.2166  -5.1973
983    Topic6    90.367340        permanently    90.549667   2.2165  -5.2461
985    Topic6    87.190765   plse_permanently    87.372650   2.2165  -5.2819
93     Topic6   451.291748  jersey_complicate   500.532043  

In [10]:
model.save('lda.model')

In [11]:
def explore_topic(lda_model, topic_number, topn, output=True):
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))   
    return terms

In [12]:
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )

term                 frequency

Topic 0 |---------------------

floral               0.029
lace                 0.024
fastening            0.023
over                 0.022
midi                 0.020
mini                 0.019
lining               0.019
wrap                 0.018
ruffle               0.017
drapes               0.016
Topic 1 |---------------------

lined                0.037
lining               0.032
more                 0.022
fully_lined          0.019
fully                0.019
close                0.019
slim                 0.017
embellhed            0.015
fastening            0.015
mesh                 0.015
Topic 2 |---------------------

short                0.045
short_sleeves        0.036
crew                 0.030
complicate           0.029
jersey_complicate    0.026
stripe               0.025
striped              0.024
true                 0.018
classic              0.017
line                 0.015
Topic 3 |---------------------

overd                0.026
rel

### Based on the above, giving a general name of each cluster.

In [13]:
top_labels = {0: 'floral', 1:'drapes', 2:'shirt', 3:'lace skirt', 4:'black bodysuit', 5:'conscious'}

In [14]:
from collections import OrderedDict
def get_doc_topic_dist(model, corpus, kwords=True):
    
    '''
    LDA transformation, for each doc only returns topics with non-zero weight
    This function makes a matrix transformation of docs in the topic space.
    '''
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(OrderedDict(tmp).values())
        top_dist += [array(vals)]
        if kwords:
            keys += [array(vals).argmax()]

    return array(top_dist), keys

In [15]:
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)


In [16]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(top_dist)

In [17]:
df1['X_tsne'] =X_tsne[:, 0]
df1['Y_tsne'] =X_tsne[:, 1]
df1['clusters'] = lda_keys

/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [18]:
cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

df1['colors'] = df1['clusters'].apply(lambda l: cluster_colors[l])

/Users/kellyho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
df1

,Unnamed: 0,alltext,brand,category,description,img_2,img_id,img_url,material,price,product_name,website,id,X_tsne,Y_tsne,clusters,colors
3136,3136,ASOS DESIGN blouse with frill shoulder Lightwe...,NaN,Top,"Lightweight smooth woven fabric,Granded collar...",NaN,NaN,https://images.asos-media.com/products/asos-de...,Main: 100% Polyester.,$40.00,ASOS DESIGN blouse with frill shoulder,ASOS,3136,50.514114,-33.524483,5,salmon
3137,3137,A Star Is Born Going Out Festival High Neck Bo...,NaN,Top,"Not just any body,High neck,Tassel trim,Hey, f...",NaN,NaN,https://images.asos-media.com/products/a-star-...,"Smooth lightweight fabric,Soft-touch style,Fab...",$108.00,A Star Is Born Going Out Festival High Neck Bo...,ASOS,3137,6.028544,5.974228,0,blue
3138,3138,Converse Cons Skate Boarding Long Sleeve T-Shi...,NaN,Top,"Printed design to chest and sleeves,The bigger...",NaN,NaN,https://images.asos-media.com/products/convers...,"Soft-touch jersey ,You can count on me ,Body: ...",$36.00,Converse Cons Skate Boarding Long Sleeve T-Shirt,ASOS,3138,-34.835716,-66.974648,3,red
3139,3139,Vans White Finish Line Heritage Long Sleeve T-...,NaN,Top,"Nothing beats a fresh tee,Crew neck,Long sleev...",NaN,NaN,https://images.asos-media.com/products/vans-wh...,"Soft-touch jersey,T-shirt fabric, in other wor...",$36.00,Vans White Finish Line Heritage Long Sleeve T-...,ASOS,3139,-59.991894,-38.772121,2,yellow
3140,3140,ASOS DESIGN x glaad& Curve t-shirt with high n...,NaN,Top,We’re partnering with GLAAD in support of the ...,NaN,NaN,https://images.asos-media.com/products/asos-de...,"Soft-touch jersey,Why complicate things?,Main:...",$35.00,ASOS DESIGN x glaad& Curve t-shirt with high n...,ASOS,3140,-87.303528,-17.238020,2,yellow
3141,3141,ASOS DESIGN Tall t-shirt with textured neon ci...,NaN,Top,"You can never have too many,Crew neck,Neon pri...",NaN,NaN,https://images.asos-media.com/products/asos-de...,"Go-with-everything jersey,It’s a soft all-roun...",$19.00,ASOS DESIGN Tall t-shirt with textured neon ci...,ASOS,3141,-77.263901,-16.120094,2,yellow
3142,3142,adidas Training All Me Vfa Bra In Black Gym mo...,NaN,Top,"Gym motivation starts with a good kit,Quick dr...",NaN,NaN,https://images.asos.com/webcontent/greenroom/v...,"Smooth stretch fabric ,Uses adidas climalite t...",$46.00,adidas Training All Me Vfa Bra In Black,ASOS,3142,-20.538416,3.043519,4,skyblue
3143,3143,Converse Long Sleeve T-Shirt In Pink Crew neck...,NaN,Top,"Crew neck,It's classic you,Converse print to c...",NaN,NaN,https://images.asos-media.com/products/convers...,"Soft-touch cotton,Nail the basics,Body: 100% C...",$30.00,Converse Long Sleeve T-Shirt In Pink,ASOS,3143,-10.669546,-87.296387,3,red
3144,3144,ASOS DESIGN Curve bardot shirred top Off-shoul...,NaN,Top,"Off-shoulder style,Long sleeves,Shirred-stretc...",NaN,NaN,https://images.asos-media.com/products/asos-de...,"Soft-touch jersey,You can count on me,Main: 10...",$29.00,ASOS DESIGN Curve bardot shirred top,ASOS,3144,56.030430,-0.358543,1,green
3145,3145,Calvin Klein Jeans box logo t shirt Wear it ag...,NaN,Top,"Wear it again and again,Crew neck,Box logo to ...",NaN,NaN,https://images.asos-media.com/products/calvin-...,"Basic jersey,Why complicate things?,Body: 100%...",$36.00,Calvin Klein Jeans box logo t shirt,ASOS,3145,-41.181053,-60.226643,3,red


In [20]:
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [21]:
source = ColumnDataSource(dict(
    x=df1['X_tsne'],
    y=df1['Y_tsne'],
    color=df1['colors'],
    label=df1['clusters'],
    topic_cluster= df1['clusters'],
    title= df1[u'product_name'],
    img_url = df1['img_url'],
    website = df1['website']
))

In [22]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source, color='color', alpha=0.8, size=10)

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Prod_name: @title, website: @website - Topic: @topic_cluster "}
plot_lda.legend.location = "top_left"

show(plot_lda)

#save the plot
save(plot_lda, '{}.html'.format(title))

/Users/kellyho/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:125: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/Users/kellyho/anaconda3/lib/python3.6/site-packages/bokeh/io/saving.py:138: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/kellyho/Desktop/capstone/Kelly/T-SNE visualization of topics.html'